# Task 2: General Hartree-Fock solver

Having set up an initial single-particle basis set $\{\chi_{\alpha}\}_{\alpha = 1}^{l}$ we now set out to find the ground state of the two-body problem.
The full Hamiltonian used by {cite}`zanghellini_2004` is given by
\begin{align}
    \hat{H}
    = \hat{h} + \hat{u}
    = \sum_{i = 1}^{n}\left(
        -\frac{1}{2}\frac{\text{d}^2}{\text{d}x^2}
        + \frac{1}{2}\omega^2 x^2
    \right)
    + \sum_{i = 1}^{n} \sum_{j = i + 1}^{n} \frac{\alpha}{\sqrt{(x_i - x_j)^2 + a^2}},
\end{align}
where the number of particles $n = 2$, $a = 0.25$, $\alpha = 1$ (not to be confused with the index $\alpha$ in the basis of harmonic oscillator eigenstates), and $\omega = 0.25$.

## Types of orbital systems

In quantum chemistry it is common to call the initial single-particle functions for _atomic orbitals_.
We will also adhere to this convention, thus making the basis of harmonic oscillator single-particle functions the atomic orbitals of our system.
Now, as the harmonic oscillator Hamiltonian is independent of spin we have restricted our view to the spatial part of the atomic orbitals when using quantum-systems to construct our basis elements.
However, when moving to the full fermionic many-body picture (two = many) we have to include spin to get the full picture.
We often talk about three kinds of orbitals depending on the restrictions put on spin.

1. The general spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        &= c_1 \varphi_1(x) \alpha(m_s) + c_2 \varphi_2(x) \beta(m_s),
    \end{align}
    where $\varphi_i(x)$ are the spatial oribtals, and $\sigma \in \{\alpha, \beta\}$ are spin basis functions with $m_s \in \{\uparrow, \downarrow\}$ the spin quantum number.
    Typical choices for the spin basis functions are
    \begin{gather}
        \alpha(\uparrow) = 1, \quad \beta(\uparrow) = 0, \\
        \alpha(\downarrow) = 0, \quad \beta(\downarrow) = 1.
    \end{gather}
2. The unrestricted spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        = \varphi_{\sigma}(x) \sigma(m_s),
    \end{align}
    where an unrestricted spin-orbital has a definite spin-direction (either $\alpha$ or $\beta$), but allows for the spatial part to be different between spin-directions.
3. The restricted spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        = \varphi(x) \sigma(m_s),
    \end{align}
    where the spin-orbitals have a definite spin-direction with the spatial part equal between the two spin-directions.

We will in this project create a general Hartree-Fock (GHF) solver which implies that we use general spin-orbitals.
This will turn out to be easiest mathematically (when deriving the equations for GHF) and the most general formulation.
However, the restricted and unrestricted formulations are more numerically efficient.